In [1]:
import numpy as np
import cantera as ct
import matplotlib.pyplot as plt
import qss_py
import time
from scipy.integrate import ode
import SundialsPy as SP
import rk_solver_cpp
import pandas as pd
from typing import Tuple, List, Dict, Any, Optional
import os
from datetime import datetime
from tqdm import tqdm
from qss_utils import integrate_single_step, create_solver

In [2]:
mechanism = "/Users/elotech/Downloads/research_code/pysundial/large_mechanism/n-dodecane.yaml"
temp = 970
pressure = 6 * ct.one_atm

fuel = 'nc12h26:1.0'
oxidizer = 'n2:3.76, o2:1.0'
phi = 0.7
gas = ct.Solution(mechanism)
gas.set_equivalence_ratio(phi, fuel, oxidizer)
gas.TP = temp, pressure
major_species = ['o', 'h', 'oh', 'h2o', 'o2', 'h2']
config = {
    'rtol': 1e-6,
    'atol': 1e-12,
    'dtmin': 1e-16,
    'dtmax': 1e-6,
    'itermax': 1,
    'stabilityCheck': False,
    'epsmin': 2e-2,
    'epsmax': 10.0,
    'abstol': 1e-11,
}




In [3]:
from numpy.matlib import True_


def benchmark_solver(solver_type, config, temperature, pressure, 
                    dt=1e-6, t_end=0.5, save_states=True):
    """
    Benchmark chemistry solvers with CFD-realistic reinitialization
    
    Parameters:
    -----------
    solver_type : str
        'cvode' or 'qss'
    config : dict
        Solver configuration
    temperature : float
        Temperature for solver
    pressure : float
        Pressure for solver
    dt : float
        Time step size (default: 1e-6)
    t_end : float
        End time (default: 0.5)
    save_states : bool
        Whether to save state history (default: True)
        
    Returns:
    --------
    dict with keys:
        'states': list of states (if save_states=True)
        'cpu_times': array of per-step CPU times
        'total_cpu_time': total CPU time
        'n_steps': number of steps
        'solver_type': solver type used
        'dt': time step used
        'final_state': final state
    """
    gas = ct.Solution(mechanism)
    gas.set_equivalence_ratio(phi, fuel, oxidizer)
    gas.TP = temperature, pressure
    
    Y0 = gas.Y.copy()
    initial_state = np.hstack(([temperature], Y0))
    n_steps = int(t_end / dt)
    
    print(f"Running {solver_type.upper()} benchmark: {n_steps} steps, dt={dt:.1e} temperature={temperature:.3f} K")
    
    # Initialize storage
    states = [] if save_states else None
    cpu_times = np.zeros(n_steps)
    current_state = initial_state.copy()
    
    # Create solver
    solver = create_solver(solver_type, config, gas, initial_state, 0.0, pressure)
    
    total_start_time = time.time()
    
    # Integration loop
    for step in tqdm(range(n_steps), desc=f'{solver_type.upper()} Integration (T={current_state[0]:.1f}K)'):
        step_start_time = time.time()
        
        if solver_type == 'cvode':
            # Reset to fresh conditions (like each CFD grid point)
            solver.set_state(current_state, 0.0)
            
            # Integrate for one time step
            try:
                result = solver.solve_to(dt)
                current_state = result
            except Exception as e:
                print(f'CVODE Error at step {step}: {e}')
                current_state = initial_state.copy()
                
        elif solver_type == 'qss':
            # Reset to fresh conditions (like each CFD grid point)
            solver.setState(current_state.tolist(), 0.0)
            
            # Integrate for one time step
            result = solver.integrateToTime(dt)
            
            if result == 0:
                current_state = np.array(solver.y)
            else:
                print(f'QSS Error at step {step}: result={result}')
                current_state = initial_state.copy()
        else:
            raise ValueError(f"Unknown solver type: {solver_type}")
        
        # Record timing and state
        step_cpu_time = time.time() - step_start_time
        cpu_times[step] = step_cpu_time
        
        if save_states:
            states.append(current_state.copy())
    
    total_cpu_time = time.time() - total_start_time
    # Prepare results
    results = {
        'states': states,
        'cpu_times': cpu_times,
        'total_cpu_time': total_cpu_time,
        'n_steps': n_steps,
        'solver_type': solver_type,
        'dt': dt,
        't_end': t_end,
        'final_state': current_state.copy(),
        'avg_step_time': np.mean(cpu_times),
        'total_step_time': np.sum(cpu_times),
        'overhead_time': total_cpu_time - np.sum(cpu_times)
    }
    
    # Print summary
    print(f"\n{solver_type.upper()} Benchmark Results:")
    print(f"  Total steps: {n_steps}")
    print(f"  Total CPU time: {total_cpu_time:.3f} s")
    print(f"  Total step time: {np.sum(cpu_times):.3f} s")
    print(f"  Overhead time: {total_cpu_time - np.sum(cpu_times):.3f} s")
    print(f"  Average step time: {np.mean(cpu_times)*1000:.3f} ms")
    print(f"  Min step time: {np.min(cpu_times)*1000:.3f} ms")
    print(f"  Max step time: {np.max(cpu_times)*1000:.3f} ms")
    print(f"  Final temperature: {current_state[0]:.3f} K")
    return results

def compare_solvers(config, T, P, dt=1e-6, t_end=0.5):
    """
    Compare CVODE and QSS solvers
    
    Returns:
    --------
    dict with 'cvode' and 'qss' results
    """
    print("Starting solver comparison...")
    print(f"Parameters: dt={dt:.1e}, t_end={t_end}")
    
    # Benchmark CVODE
    cvode_results = benchmark_solver('cvode', config,  T, P, dt, t_end, save_states=True)
    
    print("\n" + "="*50)
    
    # Benchmark QSS
    qss_results = benchmark_solver('qss', config, T, P, dt, t_end, save_states=True)
    
    # Summary comparison
    print("\n" + "="*50)
    print("COMPARISON SUMMARY:")
    print(f"CVODE total time: {cvode_results['total_cpu_time']:.3f} s")
    print(f"QSS total time:   {qss_results['total_cpu_time']:.3f} s")
    print(f"Speedup (QSS/CVODE): {cvode_results['total_cpu_time']/qss_results['total_cpu_time']:.2f}x")
    print(f"CVODE avg step: {cvode_results['avg_step_time']*1000:.3f} ms")
    print(f"QSS avg step:   {qss_results['avg_step_time']*1000:.3f} ms")
    
    return {
        'cvode': cvode_results,
        'qss': qss_results
    }



In [4]:

# # Setup your initial conditions

# dt = 1e-5
# t_end = 0.25

# # # Run single solver benchmark
# # cvode_results = benchmark_solver('cvode', config, gas, initial_state, pressure)

# # Or compare both solvers
# comparison = compare_solvers(config, temp, pressure, dt, t_end)


In [5]:
# cvode_result = comparison['cvode']
# qss_result = comparison['qss']

# cvode_states = cvode_result['final_state']
# qss_states = qss_result['final_state']

# cvode_temperature = cvode_states[0]
# qss_temperature = qss_states[0]

# print(f"CVODE temperature: {cvode_temperature:.3f} K")
# print(f"QSS temperature: {qss_temperature:.3f} K")




In [6]:
# fig, ax = plt.subplots()
# ax.plot(cvode_result['cpu_times'], label='cvode')
# ax.plot(qss_result['cpu_times'], label='qss')
# ax.legend()
# ax.set_xlabel('Time')
# ax.set_ylabel('Temperature (K)')
# ax.set_title('Temperature Comparison')

In [14]:
1925175/ct.one_atm

19.0

In [20]:
186*50*1e-6

0.0093

In [31]:
mechanism = "/Users/elotech/Downloads/research_code/pysundial/large_mechanism/n-dodecane.yaml"
temp = 819
pressure = 48 * ct.one_atm

fuel = 'nc12h26:1.0'
oxidizer = 'n2:3.76, o2:1.0'
phi = 1.21
gas = ct.Solution(mechanism)
gas.set_equivalence_ratio(phi, fuel, oxidizer)
gas.TP = temp, pressure
major_species = ['o', 'h', 'oh', 'h2o', 'o2', 'h2']
config = {
    'rtol': 1e-6,
    'atol': 1e-12,
    'dtmin': 1e-16,
    'dtmax': 1e-6,
    'itermax': 1,
    'stabilityCheck': False,
    'epsmin': 2e-2,
    'epsmax': 10.0,
    'abstol': 1e-11,
}


T0 = gas.T 
Y0 = gas.Y

solver_type = 'qss'
states = []
state = np.hstack(([T0], Y0))
cpu_times = []
solver_cpu_times = []
t0 = 0.0

solver = create_solver(solver_type, config, gas, state, t0, pressure, mxsteps=10000)


In [32]:
current_state = state.copy()

dt = 1e-6
t_end = 0.0093
n_steps = int(t_end / dt)
print(n_steps)
# solver.set_state(current_state, 0)  # Much faster than initialize()
with tqdm(total=n_steps, desc=f'Integration') as pbar:
    for step in range(n_steps):
        # Reset to fresh conditions (like each CFD grid point)
        solver.setState(current_state, 0.0) if solver_type == 'qss' else solver.set_state(current_state, 0.0)
        result = solver.solve_to(dt) if solver_type == 'cvode' else solver.integrateToTime(dt)
        current_state = np.array(result) if solver_type == 'cvode' else np.array(solver.y)
        states.append(current_state.copy())  # Store state history
        
        # Update progress bar with current temperature
        pbar.set_description(f'Integration - T={current_state[0]:.3f} K')
        pbar.update(1)

9300


Integration - T=2440.996 K: 100%|██████████| 9300/9300 [00:06<00:00, 1525.12it/s]


In [ ]:
states

In [ ]:
gas = ct.Solution('gri30.yaml')
gas.set_equivalence_ratio(1.0, 'CH4:1', 'O2:1, N2:3.76')  # Stoichiometric methane/air
gas.TP = 1200, 6 * ct.one_atm  # Higher temperature for active chemistry

T0 = gas.T
Y0 = gas.Y.copy()
state = np.hstack(([T0], Y0))

solver = create_solver('cvode', config, gas, state, 0.0, pressure)
current_state = state.copy()

dt = 1e-5
t_end = 0.01  # Shorter time for testing
n_steps = int(t_end / dt)
states = []

print(f"Starting integration: {n_steps} steps")
for step in tqdm(range(n_steps), desc=f'Integration'):
    # CFD-realistic: reset solver to fresh conditions
    solver.set_state(current_state, 0.0)  # Key fix!
    
    # Integrate for one time step
    result = solver.solve_to(dt)
    current_state = np.array(result)
    states.append(current_state.copy())
    
    if step % 100 == 0:
        print(f"Step {step}: T = {current_state[0]:.1f} K")
        print(f"Current time: {solver.get_current_time()}")



In [ ]:
(step * dt) + dt

In [ ]:
solver.get_last_step()

In [ ]:
solver.get_current_time()

In [ ]:
solver.get_stats()

In [ ]:
print(f"CVODE Final temperature: {current_state[0]:.3f} K")

In [ ]:

gas = ct.Solution('gri30.yaml')
gas.TP = 970, 6 * ct.one_atm
T0 = gas.T
Y0 = gas.Y.copy()

solver_type = 'qss'
solver = create_solver(solver_type, config, gas, state, t0, pressure)

dt = 1e-5
t_end = 0.5
n_steps = int(t_end / dt)
current_state = state

# Reset to fresh conditions (like each CFD grid point)
solver.setState(current_state.tolist(), step * dt)  # Always start from t=0
for step in tqdm(range(n_steps), desc=f'Integration - T={current_state[0]:.3f} K'):
    
    
    # Integrate for one time step duration
    result = solver.integrateToTime((step * dt) + dt)  # Integrate from 0 to dt
    
    if result == 0:
        current_state = np.array(solver.y)
    else:
        print(f'Error at step {step}')
        current_state = state
        
    if step % 10000 == 0:
        print(f"QSS temperature: {current_state[0]:.3f} K")
        
print(f"QSS Final temperature: {current_state[0]:.3f} K")




In [ ]:
state

In [ ]:
solver.y

In [ ]:
current_state

In [ ]:
i = 4
# dt = 1e-6
# t_target = i * dt

# result = solver.solve_to(t_target)
t_target = 0.9
result = solver.solve_to(t_target)

In [ ]:
solver.get_current_time()


In [ ]:
solver.get_stats()

In [ ]:
result

In [ ]:
i = 1
dt = 1e-6
t_target = i * dt
result = solver.integrateOneStep(t_target)

In [ ]:
solver.gcount

In [ ]:
solver.tn

In [ ]:
solver.tfd

In [ ]:
gas = ct.Solution('gri30.yaml')
gas.TP = 1200, 6 * ct.one_atm
T0 = gas.T
Y0 = gas.Y.copy()

t_end = 0.05
dt_output = 1e-6
n_outputs = int(t_end / dt_output)

solver_type = 'qss'
states = []
state = np.hstack(([T0], Y0))
cpu_times = []
solver_cpu_times = []
t0 = 0.0
for i in tqdm(range(1, n_outputs + 1), desc=f'Integration - T={state[0]:.3f} K'):
    t_target = i * dt_output
    start_time = time.time()
    state, cpu_time, solver_cpu_time, success = integrate_single_step(solver_type=solver_type, state=state, t0=t0, t_end=t_target, dt=dt_output, config=config, gas=gas, pressure=pressure)
    cpu_times.append(cpu_time)
    solver_cpu_times.append(solver_cpu_time)
    states.append(state)
    t0 = t_target
    print(f"QSS temperature: {state[0]:.3f} K")
    
    

In [ ]:

# for i in tqdm(range(1, n_outputs + 1), desc='Integration'):
#     t_target = i * dt_output
#     start_time = time.time()
#     result = solver.integrateToTime(t_target)
#     solver_cpu_time = time.time() - start_time
#     cpu_time = time.time() - start_time
#     if result == 0:
#         final_state = np.array(solver.y)
#         success = True
#     else:
#         final_state = state
#         success = False
#     states.append(final_state)
#     cpu_times.append(cpu_time)
#     solver_cpu_times.append(solver_cpu_time)


In [ ]:
t_end = 0.5
dt_output = 1e-6
n_outputs = int(t_end / dt_output)
temp = 970
pressure = 6 * ct.one_atm
phi = 0.7
fuel = 'CH4:1.0'
oxidizer = 'N2:3.76, O2:1.0'


def solve_traj(temp, pressure, phi, fuel, oxidizer, t_end, dt_output, solver_type, config):
    gas = ct.Solution(mechanism)
    gas.set_equivalence_ratio(phi, fuel, oxidizer)
    gas.TP = temp, pressure
    T0 = gas.T
    Y0 = gas.Y.copy()
    n_outputs = int(t_end / dt_output)
    state = np.hstack(([T0], Y0))
    t0 = 0.0
    solver = create_solver(solver_type, config, gas, state, t0, pressure)
    states = []
    cpu_times = []
    solver_cpu_times = []
    for i in tqdm(range(1, n_outputs + 1), desc='Integration'):
        if solver_type == 'qss':
            solver.setState(state.tolist(), (i-1) * dt_output)
        t_target = i * dt_output
        start_time = time.time()
        if solver_type == 'qss':
            result = solver.integrateToTime(t_target)
        else:
            result = solver.solve_single(t_target)
        solver_cpu_time = time.time() - start_time
        cpu_time = time.time() - start_time
        state = result if solver_type == 'cvode' else np.array(solver.y)
        states.append(state)
        cpu_times.append(cpu_time)
        solver_cpu_times.append(solver_cpu_time)
    print(f"Total CPU time: {np.sum(cpu_times)} - Solver CPU time: {np.sum(solver_cpu_times)}")
    return np.array(states), np.array(cpu_times), np.array(solver_cpu_times)


In [ ]:
cvode_states, cvode_cpu_times, cvode_solver_cpu_times = solve_traj(temp, pressure, phi, fuel, oxidizer, t_end, dt_output, 'cvode', config)

In [ ]:
solver.t0

In [ ]:
qss_states, qss_cpu_times, qss_solver_cpu_times = solve_traj(temp, pressure, phi, fuel, oxidizer, t_end, dt_output, 'qss', config)

In [ ]:
cvode_states.shape

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
ax1.plot(cvode_states[:, 0], label='cvode')
ax1.plot(qss_states[:, 0], label='qss')
ax1.set_xlabel('Time')
ax1.set_ylabel('Temperature (K)')
ax1.set_title('Temperature Evolution')
ax2.plot(np.log10(np.maximum(cvode_cpu_times, 1e-7)), label='cvode')
ax2.plot(np.log10(np.maximum(qss_cpu_times, 1e-7)), label='qss')
ax2.set_xlabel('Time')
ax2.set_ylabel('CPU Time (s)')
ax2.set_title('CPU Time Evolution')
ax2.legend()
ax1.legend()
plt.show()


In [ ]:

cvode_cpu_times = []
T_cvode = []
Y_cvode = []
for i in range(1, n_outputs + 1):
    t_target = i * timestep
    start_time = time.time()
    new_y = solver.solve_single(t_target)
    cvode_cpu_times.append(time.time() - start_time)
    T_cvode.append(new_y[0])
    Y_cvode.append(new_y[1:])


In [ ]:
print(np.sum(cvode_cpu_times))

In [ ]:
mechanism = 'gri30.yaml'    #"/Users/elotech/Downloads/research_code/pysundial/large_mechanism/n-dodecane.yaml"
temp = 970
pressure = 1 * ct.one_atm
gas = ct.Solution(mechanism)
fuel = 'CH4:1.0'
oxidizer = 'N2:3.76, O2:1.0'
phi = 0.7
gas.set_equivalence_ratio(phi, fuel, oxidizer)
gas.TP = temp, pressure
major_species = ['o', 'h', 'oh', 'h2o', 'o2', 'h2']

T0 = gas.T
Y0 = gas.Y.copy()

y0 = np.hstack(([T0], Y0))
t0 = 0.0
end_time = 2.5
timestep = 1e-5
n_outputs = int(end_time / timestep)
rtol = 1e-10
atol = 1e-20
species_to_track = major_species
time_limit = 300.0
table_id = None
method = 'scipy_vode_bdf'
solver = create_solver(method, gas, y0, t0, rtol, atol, end_time, pressure=pressure, table_id=table_id)


In [ ]:
scipy_cvode_cpu_times = []
T_scipy_cvode = []
Y_scipy_cvode = []
T_s = []

for i in tqdm(range(1, n_outputs + 1), desc='Scipy CVode'):
    t_target = solver.t + timestep
    start_time = time.time()
    new_y = solver.integrate(t_target)
    scipy_cvode_cpu_times.append(time.time() - start_time)
    T_scipy_cvode.append(new_y[0])
    Y_scipy_cvode.append(new_y[1:])
    T_s.append(solver.t)


In [ ]:
print(np.sum(cvode_cpu_times))

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(0, end_time, timestep)[1:len(T_cvode)+1], T_cvode, label='cvode_bdf')
ax.plot(T_s, T_scipy_cvode, label='scipy_vode_bdf')
ax.legend()
ax.set_xlabel('Time')
ax.set_ylabel('Temperature (K)')
ax.set_title('Temperature Comparison')

In [ ]:
np.sum(cvode_cpu_times)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy.integrate import ode
import cantera as ct
import warnings
 
from scipy.interpolate import interp1d
 
warnings.filterwarnings('ignore')
 
gas = ct.Solution('gri30.yaml')
species = ['T'] + gas.species_names
dt = 1e-5
N = 1000000
Tl = np.linspace(970, 1200, 1, endpoint = True, dtype=int)
#replace last entry i Tl with 980
 
phil = np.linspace(0.7, 1.5, 1, endpoint = True).round(2)
ppl = [1]
 
all_species = ['T'] + gas.species_names
inds = [all_species.index(sp) for sp in species]
 
def get_t(t, Y):
    Y= Y[:,:-2]
    sigma = (Y.max(axis = 0) - Y.min(axis = 0))/100
    t_ = [0]
    Yold = Y[0]
    for it in range(1, Y.shape[0]):
        if np.any(np.abs(Y[it] - Yold) > sigma) or t[it] - t_[-1] > 2.5e-3 * t[-1]:
            t_ += [t[it]]
            Yold = Y[it]
    t_ += [t[it]]
    return t_
 
Y0_list = []
series_num = 0
tphi = []
for kk, TT in enumerate(Tl):
    for jj, phi in enumerate(phil):
        for ii, pp in enumerate(ppl):
 
            series_num += 1            
            #if series_num != 62:
            #    continue
            tphi += [[TT, phi]]
           
       
            print("###############################################")
            print("Temperature =", TT)
            print("Equiv ratio =", phi)
            print("series number = ", series_num)
           
            gas.TP = TT, ct.one_atm*pp
            gas.set_equivalence_ratio(phi, 'CH4', 'O2:1.0, N2:3.76')
            Ytemp = gas.Y
            gas.TPY = TT, ct.one_atm*pp, np.maximum(gas.Y, 1e-16)
 
       
            #if series_num != 1:
            #    continue
           
            T_old = gas.T
            igtime = 0
            misc = False
           
            def f(t, Y):
                T = Y[0] #np.exp(Y[0])
                YY = Y[1:] #np.exp(Y[1:])
                #YY = np.maximum(YY, 1e-16)
                gas.TPY = T, ct.one_atm*pp, YY
                species_rates = gas.net_production_rates*gas.molecular_weights/gas.density
                species_h = gas.partial_molar_enthalpies/gas.molecular_weights
                temp_rate = -np.sum(species_rates*species_h/gas.cp_mass)
                return np.concatenate((np.array([temp_rate]),species_rates), axis = 0)
           
            r = ode(f).set_integrator('vode', method = 'bdf', with_jacobian=True, rtol=1e-10, atol=1e-10) #, method='bdf', with_jacobian=True)
            Y0 = np.concatenate((np.asarray(gas.TPY[0])[None], gas.Y), axis = 0)
            r.set_initial_value(Y0, 0.0)
       
            T = []
            ys = []
            ts = [0]
            for i in tqdm(range(N), desc='Scipy CVode'):
                r.integrate(r.t + dt)
                # print(i, r.y[0])
                T += [r.y[None,:]]
                # print(f'T is {T}')
                # sys.exit()
 
               
                ys += [r.y]
                ts += [r.t]
 
                # print(f'T is {T}')
                # print(f'ys is {ys}')
                # sys.exit()
                if r.y[0] > 1600 and misc == False:
                    igtime = r.t
                    misc = True
                if abs(r.y[0] - T_old) < 1 and r.y[0] > 1600 and r.t - igtime > 0.5*igtime:
                    print("steady state... quitting")
                    break
                #if r.t - igtime > 5*igtime and np.exp(r.y[0]) > 1600:
                #    print("ignited a while ago... quitting")
                #    break
                T_old = r.y[0]
            #sys.exit()
            Y0_list += [ys[0]]
       
            tc = np.concatenate(T, axis = 0)
            ys = np.stack(ys)
            species = ["TT"] + gas.species_names
           
            for i in range(0,1): #len(inds)):
                plt.plot(ts[1:], ys[:,i], '-o')
                #plt.plot(t_, ys_interp.T[:,i], '--o')
                plt.xlabel(species[inds[i]])
                plt.title("t = " + str(TT) + ", phi = " + str(phi))
                #plt.savefig('H2images/im' + str(series_num) +'.png')
                plt.show()
               
 
#             #sys.exit()
            # np.savetxt('Data/H2series/species/s' + str(series_num) + '.txt', np.hstack([t_[:, None], Y_]))
            # np.savetxt('Data/H2series/omega/omega' + str(series_num) + '.txt', omega)
            #np.savetxt('Data/H2series/P' + str(series_num) + '.txt', [pp/20])
            # print("###############################################")
#             #sys.exit()
 
# np.savetxt('Data/H2series/tphi.txt', tphi)

In [ ]:
ppl

In [ ]:

for kk, TT in enumerate(Tl):
    for jj, phi in enumerate(phil):
        for ii, pp in enumerate(ppl):
 
            series_num += 1            
            #if series_num != 62:
            #    continue
            tphi += [[TT, phi]]
           
       
            print("###############################################")
            print("Temperature =", TT)
            print("Equiv ratio =", phi)
            print("series number = ", series_num)
           
            gas.TP = TT, ct.one_atm*pp
            gas.set_equivalence_ratio(phi, 'CH4', 'O2:1.0, N2:3.76')
            Ytemp = gas.Y
            gas.TPY = TT, ct.one_atm*pp, np.maximum(gas.Y, 1e-16)
 
       
            #if series_num != 1:
            #    continue
           
            T_old = gas.T
            igtime = 0
            misc = False
           
            def f(t, Y):
                T = Y[0] #np.exp(Y[0])
                YY = Y[1:] #np.exp(Y[1:])
                #YY = np.maximum(YY, 1e-16)
                gas.TPY = T, ct.one_atm*pp, YY
                species_rates = gas.net_production_rates*gas.molecular_weights/gas.density
                species_h = gas.partial_molar_enthalpies/gas.molecular_weights
                temp_rate = -np.sum(species_rates*species_h/gas.cp_mass)
                return np.concatenate((np.array([temp_rate]),species_rates), axis = 0)
           
            r = ode(f).set_integrator('vode', method = 'bdf', with_jacobian=True, rtol=1e-10, atol=1e-10) #, method='bdf', with_jacobian=True)
            Y0 = np.concatenate((np.asarray(gas.TPY[0])[None], gas.Y), axis = 0)
            r.set_initial_value(Y0, 0.0)
       
            T = []
            ys = []
            for i in range(N):
                r.integrate(r.t + dt)
                # print(i, r.y[0])
                T += [r.y[None,:]]
                # print(f'T is {T}')
                # sys.exit()
 
               
                ys += [r.y]
 
                # print(f'T is {T}')
                # print(f'ys is {ys}')
                # sys.exit()
                if r.y[0] > 1600 and misc == False:
                    igtime = r.t
                    misc = True
                if abs(r.y[0] - T_old) < 1 and r.y[0] > 1600 and r.t - igtime > 0.5*igtime:
                    print("steady state... quitting")
                    break
                #if r.t - igtime > 5*igtime and np.exp(r.y[0]) > 1600:
                #    print("ignited a while ago... quitting")
                #    break
                T_old = r.y[0]
            #sys.exit()
            Y0_list += [ys[0]]
       
            tc = np.concatenate(T, axis = 0)
            ys = np.stack(ys)
            species = ["TT"] + gas.species_names
           
            # if TT == 950:
            #     N_ = 20 #number of points needed
            # else:
            #     N_ = 10
            N_ = 200
            t = np.linspace(0, r.t, ys.shape[0], endpoint=True)
            # print(t.shape)
            # sys.exit()
            g = interp1d(t, ys[:,inds].T, kind='linear')
            t_fine = np.linspace(0, r.t, ys.shape[0]*100)
            # print(t_fine.shape)
            #sys.exit()
            fine_y = g(t_fine).T
            tnew = get_t(t_fine, fine_y)
            t_ = np.interp(np.linspace(10e-3, r.t, N_, endpoint=True), np.linspace(10e-3, r.t, len(tnew), endpoint=True), tnew)
            # print(t_.shape)
            # sys.exit()
            #f = interp1d(t, ys.T, kind='linear')
            ys_interp = g(t_)
            print(ys_interp.shape)
            omega = np.vstack([f(t_, yss) for yss in ys_interp.T])
 
            Y_ = (ys_interp.T)#[:,inds]
            a = 50
            sn = Y_[a:,:-1]
            m,n = sn.shape
            noise = np.random.randn(m,n)
 
            # Normalize the noise to have a mean of 0 and std of 1
            noise = (noise - noise.mean()) / noise.std()
 
            # Scale the noise
            noise = 0 #noise * 0.010 * sn
 
            Y_[a:,:-1] = sn +noise
           
            for i in range(0,1): #len(inds)):
                plt.plot(t_[:100], Y_[:100,i], '-o')
                #plt.plot(t_, ys_interp.T[:,i], '--o')
                plt.xlabel(species[inds[i]])
                plt.title("t = " + str(TT) + ", phi = " + str(phi))
                #plt.savefig('H2images/im' + str(series_num) +'.png')
                plt.show()
               
 
#             #sys.exit()
            # np.savetxt('Data/H2series/species/s' + str(series_num) + '.txt', np.hstack([t_[:, None], Y_]))
            # np.savetxt('Data/H2series/omega/omega' + str(series_num) + '.txt', omega)
            #np.savetxt('Data/H2series/P' + str(series_num) + '.txt', [pp/20])
            # print("###############################################")
#             #sys.exit()
 
# np.savetxt('Data/H2series/tphi.txt', tphi)

In [ ]:
y0 = np.hstack(([T0], Y0))
t0 = 0.0
end_time = 0.04
timestep = 1e-6
n_outputs = int(end_time / timestep)
rtol = 1e-8
atol = 1e-20
species_to_track = major_species
time_limit = 300.0
table_id = None
method = 'scipy_vode_bdf'

solver = create_solver(method, gas, y0, t0, rtol, atol, end_time, pressure=pressure, table_id=table_id)

cvode_cpu_times = []
T_cvode = []
Y_cvode = []
for i in range(1, n_outputs + 1):
    t_target = i * dt_output
    start_time = time.time()
    # new_y = solver.solve_single(t_target)
    solver.integrate(t_target)
    new_y = solver.y
    cvode_cpu_times.append(time.time() - start_time)
    T_cvode.append(new_y[0])
    Y_cvode.append(new_y[1:])
    # if i % 100 == 0:
    #     print(f"Solver last step: {solver.get_last_step()}")


In [ ]:
len(T_cvode)

In [ ]:
# compare cvode_bdf and qss result

fig, axs = plt.subplots(2, 1, figsize=(10, 8))
axs[0].plot(T_cvode, label='cvode_bdf')
axs[0].plot(temps, label='qss')
axs[0].legend()
axs[1].plot(np.array(cvode_cpu_times), label='cvode_bdf')
axs[1].plot(np.array(cpu_times), label='qss')
axs[1].legend()

In [ ]:
cvode_cpu_times

In [ ]:
print(np.sum(cvode_cpu_times))

print(np.sum(cpu_times))

In [ ]:
#!/usr/bin/env python3
"""
Proper PaSR (Partially Stirred Reactor) Implementation using Cantera
Based on Lapointe et al. (2020) methodology

PaSR equation: dY_i/dt = ω_i + (Y_i,in - Y_i)/τ_res
Where:
- ω_i: chemical production rate
- Y_i,in: inlet mass fraction 
- Y_i: reactor mass fraction
- τ_res: residence time
"""

import numpy as np
import cantera as ct
import pickle
import os
from tqdm import tqdm
from scipy.integrate import solve_ivp


In [ ]:
gas